### Item 21: Use property delegation to extract common property patterns

property delegation 는 공통 property behavior 패턴을 정의하는 보편적인 방법이다.
- kotlin 에서 property 의 get, set accessors 를 다른 object 의 getValue, setValue operator 로 위임하도록 구현된다.
- property 오른쪽에 `by` 키워드 후에 위임할 object(delegates) 를 선언함으로써 사용할 수 있다. 

가장 많이 활용되는 예시가 ***lazy property*** 이다.
- lazy property 는 처음 사용될 때 초기화되는 property 이다.
```kotlin
val value by lazy { createValue() }
```

**observable property** 패턴 역시 많이 사용된다.
- property 가 바뀔 때마다 어떤 행위를 하는 property 이다.
```kotlin
var key: String? by Delegates.observable(null) { _, old, new -> 
    Log.e("key changed from $old to $new")
}

var items: List<Item> by Delgates.observable(listOf()) { _, _, _ -> 
    notifyDataSetChanged()
}
```
View and Resource Binding
```kotlin
// View and resouce binding examplel in Android
private val button: Button by bindView(R.id.button)
private val textSize by bindDimension(R.dimen.font_size)
private val doctor: Doctor by ageExtra(DOCTOR_ARG)
```

Dependency Injection
```kotlin
private val presenter: MainPresenter by inject()
private val repository: NetworkRepository by inject()
private val vm: MainViewModel by viewMdoel()
```

Data Binding
```kotlin
private val port by bindConfiguration("port")
private val token: String by preferences.bind(TOKEN_KEY)
```

In [7]:
// delgation property 를 사용하지 않으면 같은 behavior 에 대한 코드가 중복된다.
var token: String? = null
    get() {
        println("token returne value $field")
        return field
    }
    set(value) {
        println("token changed from $field to $value")
        field = value
    }

var failedLoginCount: Int = 0
    get() {
        println("failedLoginCount returned value $field")
        return field
    }
    set(value) {
        println("failedLoginCount changed from $field to $value")
        field = value
    }

failedLoginCount returned value 0
failedLoginCount returned value 0
token returne value null
token returne value null


In [8]:
// delgation property 를 사용하면 같은 behavior 에 대한 코드를 재사용할 수 있다.
import kotlin.reflect.KProperty

var token: String? by LoggingProperty(null)
var failedLoginCount: Int by LoggingProperty(0)

private class LoggingProperty<T>(var value:T) {
    operator fun getValue(thisRef: Any?, prop: KProperty<*>): T {
        println("${prop.name} returned value $value")
        return value
    }
    operator fun setValue(thisRef: Any?, prop: KProperty<*>, newValue: T) {
        println("${prop.name} changed from $value to $newValue")
        value = newValue
    }
}

failedLoginCount returned value 0
failedLoginCount returned value 0
token returned value null
token returned value null
token returned value null
failedLoginCount returned value 0
token returned value null


The problem is found in one of the loaded libraries: check library converters (fields callbacks)
org.jetbrains.kotlinx.jupyter.exceptions.ReplEvalRuntimeException: 
org.jetbrains.kotlinx.jupyter.exceptions.ReplLibraryException: The problem is found in one of the loaded libraries: check library converters (fields callbacks)
	at org.jetbrains.kotlinx.jupyter.exceptions.CompositeReplExceptionKt.throwLibraryException(CompositeReplException.kt:52)
	at org.jetbrains.kotlinx.jupyter.codegen.FieldsProcessorImpl.process(FieldsProcessorImpl.kt:68)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$1.invoke(CellExecutorImpl.kt:98)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$1.invoke(CellExecutorImpl.kt:97)
	at org.jetbrains.kotlinx.jupyter.config.LoggingKt.catchAll(Logging.kt:77)
	at org.jetbrains.kotlinx.jupyter.config.LoggingKt.catchAll$default(Logging.kt:71)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute(CellExecutorImpl.kt:9

getValue, setValue 컴파일 된 코드를 보면 property 에 대한 bounded reference 와 context(`this`, 위 코드에서 `thisRef` 부분) 를 주입 받는다.
- bounded reference 를 통해서, property 의 name 이나 annotation 에 대한 정보를 받아볼 수 있다.
- context 를 통해서, function 이 어디서 사용되는지 알 수 있다.

getValue, setValue 를 context type 을 달리해서 overloading 이 가능하다.
```kotlin
class SwipeRefreshBinderDelegate(val id: Int) {
    private var cache: SwipeRefreshLayout? = null

    operator fun getValue(activity: Activity, prop: KProperty<*>) {
        // ...
    }

    operator fun getValue(fragment: Fragment, prop: KProperty<*>) {
        // ...
    }
}
```